## 1. Setup & Imports

In diesem Schritt werden die benötigten Libraries importiert und die Dateipfade definiert.
Die Daten liegen außerhalb des Notebooks im `/data` Ordner — dadurch bleibt das Notebook portabel
und die Rohdaten werden nicht versehentlich versioniert.

> ℹ️ **Disclaimer:**  
> Dieses Notebook dient primär der **explorativen Datenanalyse (EDA)** und der **Dokumentation der Gedankengänge** im Analyseprozess.  
> 
> Es ist **nicht für den produktiven Einsatz oder die Automatisierung** vorgesehen.  
> Daher wurde bewusst auf Elemente wie `try/except`-Blöcke oder generische Fehlerbehandlung verzichtet.  
> 
> Ebenso wurden **Performance-Optimierungen** bewusst ausgelassen,  
> da das Datenvolumen überschaubar ist und der Fokus auf **Nachvollziehbarkeit und Transparenz der Analyse** liegt.  


In [2]:
from pathlib import Path
import pandas as pd
from IPython.display import display
import numpy as np

# Data directory relativ zu notebooks/
DATA_DIR = Path("../data")

umsatz_file = DATA_DIR / "Umsatzdaten.xlsx"
tracking_file = DATA_DIR / "Trackingdaten.xlsx"

umsatz_file, tracking_file


(WindowsPath('../data/Umsatzdaten.xlsx'),
 WindowsPath('../data/Trackingdaten.xlsx'))

## 2. Laden der Rohdaten

Die Excel-Dateien werden eingelesen:
- `Umsatzdaten.xlsx`: enthält Bestellungen/Umsätze
- `Trackingdaten.xlsx`: enthält Klick / Bewerbung Tracking

Beide werden als pandas DataFrames eingelesen und ausgegeben, um einen ersten Eindruck zu gewinnen.

In [3]:
df_sales = pd.read_excel(umsatz_file)
df_track = pd.read_excel(tracking_file)

display(df_sales.head(2))
display(df_track.head(2))



,DATE,POSITION_NUMBER,BC_NUMBER,PRODUCT_NAME,TARIFF_NAME,COMPANY_NAME,HEADLINE,SALE_PRICE
0,2024-12-15 00:00:00,245329553AC,245329553A3,indeed.com,30 Tage / 360 € Budget,Medizinisches Zentrum Süd,IT-Sicherheitsbeauftragte / IT-Sicherheitsbeau...,360.0
1,2024-12-15 00:00:00,215429626AA,215429626A9,indeed.com,30 Tage / 360 € Budget,Medizinisches Zentrum Süd,MTA (w/m/d),360.0


,DATE,BC_NUMBER,SUPPLIER,VIEWS
0,2025-07-22 00:00:00,284281037A3,StepStone,195
1,2025-02-05 00:00:00,203293100A2,indeed,134


### 3. Struktur & Datentypen prüfen

Bevor wir mit der Bereinigung starten, prüfen wir:

- welche Spalten vorhanden sind
- wie viele Zeilen / Speicherbedarf
- welche Datentypen Pandas erkannt hat

➡️ Ziel: Verständnis über das Rohformat, bevor wir Felder konvertieren.

In [4]:
df_sales.info()
print("-" * 60)
df_track.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1611 entries, 0 to 1610
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   DATE             1611 non-null   object 
 1   POSITION_NUMBER  1611 non-null   object 
 2   BC_NUMBER        1611 non-null   object 
 3   PRODUCT_NAME     1611 non-null   object 
 4   TARIFF_NAME      1611 non-null   object 
 5   COMPANY_NAME     1611 non-null   object 
 6   HEADLINE         1611 non-null   object 
 7   SALE_PRICE       1611 non-null   float64
dtypes: float64(1), object(7)
memory usage: 100.8+ KB
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57127 entries, 0 to 57126
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   DATE       57127 non-null  object
 1   BC_NUMBER  57127 non-null  object
 2   SUPPLIER   57127 non-null  object
 3   VIEWS      

➡️ Ergebnis: Die meisten Felder wurden als `object` eingelesen (= Excel-typisch).
Im nächsten Schritt konvertieren wir Datumsfelder und prüfen fehlende Werte & Duplikate.

##  4. Data Quality & Preprocessing 
Wir prüfen und bereinigen in klaren Schritten. Nichts wird „blind“ gelöscht; jede Entscheidung ist dokumentiert.

In [5]:

df_sales_clean = df_sales.copy()
df_track_clean = df_track.copy()


### 4.1 Datumsfelder sicher konvertieren
DATE muss für Zeitlogik nutzbar sein. Fehlerhafte Werte werden als NaT markiert (keine Exceptions).

In [6]:
for df in (df_sales_clean, df_track_clean):
    df["DATE_raw"] = df["DATE"]  # Originalwert sichern
    df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce")

## 4. Data Quality Checks

> ℹ️ **Ziel:**  
> In diesem Schritt prüfen wir die Datenqualität, insbesondere fehlende Werte.  
> Zuerst erfolgt eine Gesamtübersicht pro Spalte, anschließend eine detaillierte Betrachtung nach Quelle (`SUPPLIER` bzw. `COMPANY_NAME`),  
> um Muster und potenzielle Datenherkunftsprobleme zu erkennen.

In [7]:
# Übersicht fehlender Werte pro Spalte
print("Missing per column – Sales")
display(df_sales_clean.isna().sum())

print("\nMissing per column – Tracking")
display(df_track_clean.isna().sum())


# --- Detailanalyse: Fehlende DATE-Werte nach SUPPLIER ---
print("\nTracking: fehlendes DATE nach SUPPLIER")

df_missing_tracking = (
    df_track_clean
    .assign(is_missing=df_track_clean["DATE"].isna())
    .groupby("SUPPLIER", dropna=False)
    .agg(
        total=("DATE", "size"),
        missing=("is_missing", "sum")
    )
    .assign(missing_pct=lambda x: (x["missing"] / x["total"] * 100).round(1))
    .sort_values("missing", ascending=False)  # Sortierung nach absolutem Impact
    .query("missing>0")
)

display(df_missing_tracking)

# --- Optional: Gleiche Logik für Sales-Daten ---
if "COMPANY_NAME" in df_sales_clean.columns:
    print("\nSales: fehlendes DATE nach COMPANY_NAME")

    df_missing_sales = (
        df_sales_clean
        .assign(is_missing=df_sales_clean["DATE"].isna())
        .groupby("COMPANY_NAME", dropna=False)
        .agg(
            total=("DATE", "size"),
            missing=("is_missing", "sum")
        )
        .assign(missing_pct=lambda x: (x["missing"] / x["total"] * 100).round(1))
        .sort_values("missing", ascending=False)
        .query("missing>0")
    )

    display(df_missing_sales.head(10))


Missing per column – Sales


DATE               2
POSITION_NUMBER    0
BC_NUMBER          0
PRODUCT_NAME       0
TARIFF_NAME        0
COMPANY_NAME       0
HEADLINE           0
SALE_PRICE         0
DATE_raw           0
dtype: int64


Missing per column – Tracking


DATE         277
BC_NUMBER      0
SUPPLIER       0
VIEWS          0
DATE_raw       0
dtype: int64


Tracking: fehlendes DATE nach SUPPLIER


,total,missing,missing_pct
SUPPLIER,,,
StepStone,25942,122,0.5
jobware,5953,29,0.5
indeed,5299,29,0.5
stellenanzeigen,4499,24,0.5
academicsde,3770,22,0.6
monster,2075,11,0.5
opportunoDE,837,4,0.5
ingenieurkarriere,1091,4,0.4
aerztestellenDE,546,3,0.5



Sales: fehlendes DATE nach COMPANY_NAME


,total,missing,missing_pct
COMPANY_NAME,,,
Fachhochschule Mitte,44,1,2.3
Kommunalverwaltung Ost,20,1,5.0


> 🔍 **Interpretation:**  
> Die Tabelle ist nach absoluter Anzahl fehlender `DATE`-Werte sortiert.  
> So werden die **größten Problemquellen** zuerst sichtbar (z. B. Lieferanten mit vielen Einträgen, bei denen das Datum fehlt).  
> 
> Der Prozentwert (`missing_pct`) ergänzt diese Sicht, um zu erkennen,  
> ob das Problem flächendeckend (hoher Anteil) oder nur punktuell (geringer Anteil) auftritt.

> 📈 **Erkenntnisse & nächste Schritte:**  
> Die Analyse zeigt, dass insbesondere große Anbieter wie `StepStone`, `jobware` und `indeed` den Großteil der fehlenden `DATE`-Werte verursachen.  
> Obwohl der relative Anteil gering ist (~0.5 %), ist der **absolute Impact** aufgrund der hohen Datenmenge relevant.  
>
> In den nächsten Schritten soll daher eine **gezielte Ursachenanalyse** für die Top-Verursacher erfolgen,  
> z. B. um zu prüfen, ob die fehlenden Datumswerte systematisch sind (z. B. bestimmte Wochen, Kampagnen oder Datenquellen).  
>
> Anbieter mit nur vereinzelten Fehlwerten (< 5 Fälle) werden zunächst **nicht weiter untersucht**,  
> da deren Einfluss auf die Gesamtanalyse als vernachlässigbar einzuschätzen ist.

# Ungültige Datumswerte im Umsatzdataset
invalid_sales = df_sales_clean.loc[df_sales_clean["DATE"].isna(), "DATE_raw"].unique()
print("❗ Ungültige Datumseinträge in Umsatzdaten:", invalid_sales)

# Ungültige Datumswerte im Trackingdataset
invalid_track = df_track_clean.loc[df_track_clean["DATE"].isna(), "DATE_raw"].unique()
print("❗ Ungültige Datumseinträge in Trackingdaten:", invalid_track)

print("\n📌 Insight:")
print(
    "Die oben gezeigten Werte konnten nicht in ein gültiges Datumsformat konvertiert werden.\n"
    "Auffällig ist, dass sowohl im Umsatz- als auch im Trackingdataset der Wert '29.02.2025'\n"
    "auftritt. Da das Jahr 2025 kein Schaltjahr ist, ist der 29. Februar ein nicht existierendes\n"
    "Datum – dies führt bei der Konvertierung entsprechend zu NaT.\n"
)


In [8]:
# Ungültige Datumswerte im Umsatzdataset
invalid_sales = df_sales_clean.loc[df_sales_clean["DATE"].isna(), "DATE_raw"].unique()
print("❗ Ungültige Datumseinträge in Umsatzdaten:", invalid_sales)

# Ungültige Datumswerte im Trackingdataset
invalid_track = df_track_clean.loc[df_track_clean["DATE"].isna(), "DATE_raw"].unique()
print("❗ Ungültige Datumseinträge in Trackingdaten:", invalid_track)

print("\n📌 Insight:")
print(
    "Die oben gezeigten Werte konnten nicht in ein gültiges Datumsformat konvertiert werden.\n"
    "Auffällig ist, dass sowohl im Umsatz- als auch im Trackingdataset der Wert '29.02.2025'\n"
    "auftritt. Da das Jahr 2025 kein Schaltjahr ist, ist der 29. Februar ein nicht existierendes\n"
    "Datum – dies führt bei der Konvertierung entsprechend zu NaT.\n"
)


❗ Ungültige Datumseinträge in Umsatzdaten: ['29.02.2025']
❗ Ungültige Datumseinträge in Trackingdaten: ['29.02.2025']

📌 Insight:
Die oben gezeigten Werte konnten nicht in ein gültiges Datumsformat konvertiert werden.
Auffällig ist, dass sowohl im Umsatz- als auch im Trackingdataset der Wert '29.02.2025'
auftritt. Da das Jahr 2025 kein Schaltjahr ist, ist der 29. Februar ein nicht existierendes
Datum – dies führt bei der Konvertierung entsprechend zu NaT.



### Schlussfolgerung

Die Untersuchung zeigt, dass alle NaT-Werte in beiden Datensätzen auf denselben
fehlerhaften Rohwert zurückzuführen sind:

29.02.2025 — ein Datum, das nicht existiert, da 2025 kein Schaltjahr ist.

Das bedeutet:

Der Fehler ist systematisch und tritt in beiden Quellen auf

Mögliche Ursache: Eingabefehler, Exportfehler, oder ein Platzhalterdatum für „Ende Februar“

Die Anzahl der betroffenen Zeilen ist sehr gering → geringer Einfluss aufs Gesamtergebnis

Für ein späteres Data Cleaning ergeben sich folgende Optionen:

Mapping auf ein gültiges Datum (z. B. 28.02.2025), falls der Tag wichtig ist

Interpretation auf Monatsebene (Februar 2025), falls der exakte Tag irrelevant ist

Droppen der Zeilen, falls der Anteil vernachlässigbar ist

Oder Einbau einer Datenqualitäts-Flag für Transparenz

Im aktuellen EDA-Schritt ist jedoch lediglich wichtig,
den Fehler zu erkennen und korrekt einzuordnen, ohne ihn bereits zu beheben.

### Plausibilitätscheck der Datenqualität

Nachdem die problematischen Datumswerte bereits identifiziert wurden, führen wir jetzt einen zusätzlichen Plausibilitätscheck durch, um sicherzustellen, dass beide Datensätze inhaltlich konsistent und zuverlässig nutzbar sind.

Der Fokus liegt auf:

Dubletten (ganze Zeilen)

Dubletten in Schlüsselspalten (z. B. BC_NUMBER)

Fehlenden Werten

Unplausiblen numerischen Werten

Auffälligen strukturellen Mustern

Cross-Dataset-Konsistenz (nur vorbereitend)

Dieser Schritt dient dazu, mögliche Datenqualitätsprobleme frühzeitig zu erkennen, bevor wir darauf basierende KPIs oder BI-Modelle bauen.

In [10]:
def plausi_check(df, title):
    print(f"\n===== {title.upper()} =====")
    
    dup_rows = pd.DataFrame({"duplicate_rows": [df.duplicated().sum()]})
    
    if "BC_NUMBER" in df.columns:
        dup_bc = pd.DataFrame({"duplicate_BC_NUMBER": [df["BC_NUMBER"].duplicated().sum()]})
    else:
        dup_bc = pd.DataFrame({"duplicate_BC_NUMBER": ["n/a"]})
    
    
    numeric_summary = df.select_dtypes(include=["int64","float64"]).describe()
    
    
    empty = {
        col: (df[col].astype(str).str.strip() == "").sum()
        for col in df.select_dtypes(include="object").columns
    }
    empty_df = pd.DataFrame.from_dict(empty, orient="index", columns=["empty_strings"])
    
    # ---- DISPLAY CLEAN ----   
    print("\n🔹 Duplicate Rows:")
    display(dup_rows)
    
    print("\n🔹 Duplicate BC_NUMBER:")
    display(dup_bc)
    
    print("\n🔹 Numeric Summary:")
    display(numeric_summary)
    
    print("\n🔹 Empty String Checks:")
    display(empty_df)

# Run for both dataframes
plausi_check(df_sales_clean, "Umsatzdaten")
plausi_check(df_track_clean, "Trackingdaten")




===== UMSATZDATEN =====

🔹 Duplicate Rows:


,duplicate_rows
0,0



🔹 Duplicate BC_NUMBER:


,duplicate_BC_NUMBER
0,820



🔹 Numeric Summary:


...

,duplicate_BC_NUMBER
0,56541



🔹 Numeric Summary:


,VIEWS
count,57127.000000
mean,207.106395
std,241.426636
min,1.000000
25%,46.000000
50%,153.000000
75%,286.000000
max,2648.000000



🔹 Empty String Checks:


,empty_strings
BC_NUMBER,0
SUPPLIER,0
DATE_raw,0


### Duplikate & Struktur
Umsatzdaten

Keine vollständigen Duplikate

BC_NUMBER kommt mehrfach vor → eine BC_NUMBER steht für mehrere Positionen (logisch).

Trackingdaten

Ein vollständiges Duplikat (vernachlässigbar)

BC_NUMBER kommt sehr häufig vor → entspricht der Natur von View-Events (mehrere Aufrufe pro Anzeige).

Insight:
Zwischen Umsatz- und Trackingdaten besteht ein many-to-many Verhältnis über BC_NUMBER.
Für spätere Analysen müssen beide Seiten zunächst pro BC_NUMBER aggregiert werden (z. B. Umsatz summieren, Views summieren).

### Datumswerte

In beiden Datensätzen tritt derselbe ungültige Wert auf:

29.02.2025 – nicht gültig, da 2025 kein Schaltjahr ist.

Dies führt beim Konvertieren zu NaT.
Der Fehler ist geringfügig, aber sollte später gezielt korrigiert oder vereinheitlicht werden.

### Fazit

Die Daten sind insgesamt gut strukturiert und stabil.
Einziger relevanter Punkt ist der Schaltjahrfehler in den Datumswerten sowie die notwendige Aggregation über BC_NUMBER, bevor die Datensätze sinnvoll zusammengeführt oder analysiert werden können.

### Korrektur ungültiger Datumswerte

Bei der Prüfung der DATE-Spalten fiel in beiden Datensätzen derselbe ungültige Wert auf:

29.02.2025 – nicht existent, da 2025 kein Schaltjahr ist.

Damit die Daten für die weitere Analyse sauber genutzt werden können, wird dieser Wert konsistent auf ein gültiges Datum gemappt:

➡️ 28.02.2025

Das Mapping betrifft nur wenige Zeilen und stellt sicher, dass später keine NaT-Werte entstehen und Zeitreihen sauber ausgewertet werden können.

In [ ]:
# Ungültiges Datum in beiden DFs auf gültiges Datum setzen
for df in (df_sales_clean, df_track_clean):
    df.loc[df["DATE_raw"] == "29.02.2025", "DATE"] = pd.to_datetime("2025-02-28")




(np.int64(0), np.int64(0))

## Weiter mit der Analyse

Nach Korrektur der Datumswerte und Klärung der Datenstruktur können die Datensätze jetzt ohne Einschränkungen analysiert werden.

Die nächsten Schritte konzentrieren sich auf:

Zeitliche Struktur (Monate, Zeiträume, Trends)

Umsatz- und Kundenanalyse

Inaktivitätserkennung

BC_NUMBER-Aggregation als Vorbereitung für Power BI

Damit schaffen wir eine solide Grundlage für Reporting, Visualisierung und spätere KPI-Definitionen.

## Zeitliche Struktur der Daten

Im ersten Schritt wird geprüft, welche Zeiträume die Daten abdecken und wie die zeitliche Verteilung aussieht. 
Dies ist wichtig für:
- spätere Monatsanalysen,
- Trendbewertungen,
- und die Ableitung von Inaktivität (z. B. Kunde ohne Umsatz seit X Monaten).


In [13]:
# Zeitliche Übersicht für beide Datensätze
def time_overview(df, name):
    print(f"\n===== {name} =====")
    print("Min DATE:", df["DATE"].min())
    print("Max DATE:", df["DATE"].max())
    print("\nAnzahl Einträge pro Monat:")
    display(df["DATE"].dt.to_period("M").value_counts().sort_index())

time_overview(df_sales_clean, "Umsatzdaten")
time_overview(df_track_clean, "Trackingdaten")



===== Umsatzdaten =====
Min DATE: 2024-08-21 00:00:00
Max DATE: 2025-10-11 00:00:00

Anzahl Einträge pro Monat:


DATE
2024-08     59
2024-09    163
2024-10    188
2024-11    139
2024-12    142
2025-01    160
2025-02    117
2025-03    111
2025-04    127
2025-05    114
2025-06    112
2025-07    108
2025-08     64
2025-09      3
2025-10      4
Freq: M, Name: count, dtype: int64


===== Trackingdaten =====
Min DATE: 2024-08-21 00:00:00
Max DATE: 2025-07-31 00:00:00

Anzahl Einträge pro Monat:


DATE
2024-08      120
2024-09      531
2024-10      551
2024-11     2027
2024-12     1877
2025-01     3954
2025-02     5152
2025-03     5430
2025-04     6690
2025-05     5046
2025-06     8355
2025-07    17394
Freq: M, Name: count, dtype: int64

## Umsatzanalyse

Im nächsten Schritt wird der Umsatz zeitlich und nach Kunden analysiert. 
Diese Basis-Insights sind wichtig für:
- monatliche KPIs,
- Kundenranking,
- und spätere Inaktivitätsbewertungen.


In [14]:
# Umsatz pro Monat
umsatz_monat = (
    df_sales_clean
    .groupby(df_sales_clean["DATE"].dt.to_period("M"))["SALE_PRICE"]
    .sum()
    .reset_index()
    .rename(columns={"DATE": "MONTH", "SALE_PRICE": "TOTAL_REVENUE"})
)

display(umsatz_monat)


,MONTH,TOTAL_REVENUE
0,2024-08,47094.96
1,2024-09,110123.71
2,2024-10,114386.49
3,2024-11,89219.19
4,2024-12,88183.02
5,2025-01,140866.99
6,2025-02,94637.00
7,2025-03,74735.00
8,2025-04,83929.70
9,2025-05,81764.00


## Umsatz pro Kunde

Um ein erstes Verständnis über die Geschäftsentwicklung zu erhalten, 
wird der Gesamtumsatz pro Kunde berechnet. Dadurch lassen sich umsatzstarke 
und umsatzschwache Kunden identifizieren, was später für Ranking, 
Segmentierung und Inaktivitätsanalyse wichtig ist.


In [15]:
# Umsatz pro Kunde
umsatz_kunde = (
    df_sales_clean
    .groupby("COMPANY_NAME")["SALE_PRICE"]
    .sum()
    .reset_index()
    .rename(columns={"SALE_PRICE": "TOTAL_REVENUE"})
    .sort_values("TOTAL_REVENUE", ascending=False)
)

display(umsatz_kunde.head(10))


,COMPANY_NAME,TOTAL_REVENUE
14,Medizinisches Zentrum Süd,443980.00
3,Bildungsinstitut West,256245.62
10,Kommunalverwaltung Nord,204388.08
9,Katholische Kliniken GmbH,48456.00
5,Fachhochschule Mitte,44726.00
13,Medizinisches Zentrum Mitte,44652.01
6,Fraktionsbüro Bündnis - Personalabteilung -,40587.99
11,Kommunalverwaltung Ost,17947.98
17,Steuerberatung & Partner GmbH,7117.00
4,Desinfektionslösungen GmbH,5592.00


## Letzte Aktivität pro Kunde

Um aktive von inaktiven Kunden unterscheiden zu können, wird für jeden 
Kunden das letzte Umsatzdatum ermittelt. Zusätzlich wird das jüngste 
Trackingdatum je BC_NUMBER bzw. Kunde betrachtet.

Diese Kennzahlen sind später die Basis, um Kunden nach Aktivität 
zu segmentieren (z. B. aktiv, gefährdet, inaktiv).


In [16]:
# Letztes Umsatzdatum pro Kunde
letzter_umsatz = (
    df_sales_clean
    .groupby("COMPANY_NAME")["DATE"]
    .max()
    .reset_index()
    .rename(columns={"DATE": "LAST_SALE_DATE"})
)

display(letzter_umsatz.head())


,COMPANY_NAME,LAST_SALE_DATE
0,Autohandel West GmbH & Co. KG,2025-07-02
1,Bauunternehmen Mitte GmbH,2025-01-18
2,Bildungsakademie GmbH,2025-01-26
3,Bildungsinstitut West,2025-10-11
4,Desinfektionslösungen GmbH,2024-09-11


In [ ]:
#Prüfen, ob BC_NUMBER eindeutig einem Kunden zugeordnet ist
bc_to_customer = (
    df_sales_clean
    .groupby("BC_NUMBER")["COMPANY_NAME"]
    .nunique()
    .reset_index()
    .rename(columns={"COMPANY_NAME": "N_CUSTOMERS"})
)

# BC_NUMBERs mit mehr als einem Kunden (Konflikte)
mapping_conflicts = bc_to_customer.query("N_CUSTOMERS > 1")

display(mapping_conflicts.head())
print("Anzahl Konflikte:", len(mapping_conflicts))


,BC_NUMBER,N_CUSTOMERS


Anzahl Konflikte: 0


Erwartung / Insight: Anzahl Konflikte: 0 → jede BC_NUMBER gehört genau einem Kunden.

In [31]:
bc_customer_map = (
    df_sales_clean[["BC_NUMBER", "COMPANY_NAME"]]
    .drop_duplicates()
)

display(bc_customer_map)

,BC_NUMBER,COMPANY_NAME
0,245329553A3,Medizinisches Zentrum Süd
1,215429626A9,Medizinisches Zentrum Süd
2,215230305A3,Fachhochschule Mitte
3,295234930A3,Medizinisches Zentrum Süd
4,275335938A2,Bildungsinstitut West
...,...,...
1586,244471777A9,Medizinisches Zentrum Süd
1587,224375272A3,Medizinisches Zentrum Süd
1592,234398602A2,Medizinisches Netzwerk GmbH & Co. KG
1603,287314884A4,Medizinisches Zentrum Süd


In [32]:
# 3) Trackingdaten um Kundennamen erweitern
df_track_mapped = df_track_clean.merge(
    bc_customer_map,
    on="BC_NUMBER",
    how="left"
)

display(df_track_mapped.head())


,DATE,BC_NUMBER,SUPPLIER,VIEWS,DATE_raw,COMPANY_NAME
0,2025-07-22,284281037A3,StepStone,195,2025-07-22 00:00:00,Kommunalverwaltung Nord
1,2025-02-05,203293100A2,indeed,134,2025-02-05 00:00:00,Medizinisches Zentrum Mitte
2,2025-01-21,284244442A4,stellenanzeigen,263,2025-01-21 00:00:00,Medizinisches Zentrum Mitte
3,2025-07-14,235418859A5,indeed,90,2025-07-14 00:00:00,Medizinisches Zentrum Süd
4,2025-01-18,264347912A7,StepStone,149,2025-01-18 00:00:00,Kommunalverwaltung Nord


In [34]:
# 4) Letztes Trackingdatum pro Kunde
letztes_tracking = (
    df_track_mapped
    .groupby("COMPANY_NAME")["DATE"]
    .max()
    .reset_index()
    .rename(columns={"DATE": "LAST_TRACK_DATE"})
)

display(letztes_tracking)


,COMPANY_NAME,LAST_TRACK_DATE
0,Autohandel West GmbH & Co. KG,2025-07-31
1,Bauunternehmen Mitte GmbH,2025-07-31
2,Bildungsakademie GmbH,2025-07-31
3,Bildungsinstitut West,2025-07-31
4,Desinfektionslösungen GmbH,2025-07-31
5,Fachhochschule Mitte,2025-07-31
6,Fraktionsbüro Bündnis - Personalabteilung -,2025-07-31
7,Holdinggesellschaft Nord GmbH,2025-07-31
8,Institut für Bildungsforschung und Qualität,2025-07-31
9,Katholische Kliniken GmbH,2025-07-31


In [43]:
df_track_mapped["COMPANY_NAME"].isna().sum()

np.int64(1510)

In [44]:
# Alle Tracking-Zeilen zu diesem Kunden ansehen
df_track_mapped[df_track_mapped["COMPANY_NAME"] == kunde] \
    .sort_values("DATE") \
    .tail(20)

,DATE,BC_NUMBER,SUPPLIER,VIEWS,DATE_raw,COMPANY_NAME
24868,2025-07-24,276220922A4,StepStone,354,2025-07-24 00:00:00,Autohandel West GmbH & Co. KG
10789,2025-07-24,294271228A3,StepStone,163,2025-07-24 00:00:00,Autohandel West GmbH & Co. KG
42973,2025-07-24,255370696A0,StepStone,326,2025-07-24 00:00:00,Autohandel West GmbH & Co. KG
13042,2025-07-24,226482478A5,StepStone,225,2025-07-24 00:00:00,Autohandel West GmbH & Co. KG
33710,2025-07-24,207497216A6,StepStone,196,2025-07-24 00:00:00,Autohandel West GmbH & Co. KG
1730,2025-07-25,294271228A3,StepStone,163,2025-07-25 00:00:00,Autohandel West GmbH & Co. KG
43145,2025-07-25,207497216A6,StepStone,202,2025-07-25 00:00:00,Autohandel West GmbH & Co. KG
11332,2025-07-25,276220922A4,StepStone,354,2025-07-25 00:00:00,Autohandel West GmbH & Co. KG
5005,2025-07-25,255370696A0,StepStone,326,2025-07-25 00:00:00,Autohandel West GmbH & Co. KG
15766,2025-07-25,226482478A5,StepStone,225,2025-07-25 00:00:00,Autohandel West GmbH & Co. KG


In [45]:
missing_bc = set(df_track_clean["BC_NUMBER"]) - set(df_sales_clean["BC_NUMBER"])
len(missing_bc)


15

In [46]:
# Zeilen im gemergten Tracking, bei denen CUSTOMER_NAME fehlt
na_rows = df_track_mapped[df_track_mapped["COMPANY_NAME"].isna()]

# Welche BC_NUMBERs haben diese NaN-Zeilen?
na_bc_numbers = set(na_rows["BC_NUMBER"])

# Test: Ist die Set-Difference EXACT gleich?
na_bc_numbers == missing_bc


True

In [47]:
# Erwartete Anzahl Zeilen aus Trackingdataset,
# die zu den nicht-vorhandenen BC_NUMBERs gehören
expected_na_count = df_track_clean[df_track_clean["BC_NUMBER"].isin(missing_bc)].shape[0]

expected_na_count, df_track_mapped["COMPANY_NAME"].isna().sum()


(1510, np.int64(1510))